In [9]:
import numpy as np 
import pandas as pd
import re 
import os

In [10]:
df = pd.read_csv(r"C:\Users\S1130866\Desktop\Product_Automate\product.csv")
products = [i for i in set(df.Products.str.lower().tolist()) if not isinstance(i, (int, float)) and i != '0']

In [11]:
## Reading the main file that was created from Final_File_Import_Export_Version2

df_main = pd.read_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Compiled_File\Main.xlsx")


In [12]:
df_main.head()

,Unnamed: 0,Type,Date,IMPORTER/EXPORTER NAME,Country,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Molecule Name,Product Name,Avg_Price
0,0,Import,2021-01-15,KRISHI RASAYAN EXPORTS PRIVATE LIMITED,CHINA,EMAMECTIN BENZOATE 95% TECHNICAL.ACTUAL USER R...,3000.0,KGS,168.000000,504000.0000,USD,HUNAN GOFAR FINE CHEMICAL INDUSTR,EMAMECTIN,EMAMECTIN TECH 95%,168.000000
1,1,Import,2021-01-28,WILLOWOOD CHEMICALS PVT LTD,CHINA,EMAMECTIN BENZOATE 95% TECH (CIB NO. CIR-1976/...,5000.0,KGS,155.240198,776200.9898,CNY,WILLOWOOD HANGZHOU CO LIMITED,EMAMECTIN,EMAMECTIN TECH 95%,155.240198
2,2,Import,2021-02-05,SYNGENTA INDIA LIMITED,AUSTRIA,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,1800.0,KGS,190.296299,342533.3379,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,EMAMECTIN Others,190.296299
3,3,Import,2021-02-19,SYNGENTA INDIA LIMITED,AUSTRIA,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,2250.0,KGS,190.554503,428747.6312,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,EMAMECTIN Others,190.554503
4,4,Import,2021-03-04,CRYSTAL CROP PROTECTION LIMITED,CHINA,EMAMECTIN BENZOATE TECHNICAL 95% (MIN) (CIB NO...,3000.0,KGS,228.500000,685500.0000,USD,HEBEI VEYONG BIO CHEMICAL CO LIMITED,EMAMECTIN,EMAMECTIN TECH 95%,228.500000


In [13]:
df_main.drop('Unnamed: 0',axis =1 , inplace = True)

### --------------------------------------Taking input from the user for the Export file----------------------------------------------------------------

In [14]:
# Taking the file path input from the user 

file_path = input('Enter the 38 Series Export file path: ')

# e.g. C:\Users\Bob\Desktop\example.txt
# or /home/Bob/Desktop/example.txt
print(file_path)

if os.path.exists(file_path):
    print('The file exists')

    with open(file_path, 'r', encoding="latin-1") as f:
        lines = f.readlines()

else:
    print('The specified file does NOT exist')
    

Enter the 38 Series Export file path: C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\January 2023\Export\38_Exp_TY2_Jan-23.xlsx
C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\January 2023\Export\38_Exp_TY2_Jan-23.xlsx
The file exists


In [15]:
## Reading the file
df2 = pd.read_excel(file_path)

In [16]:


# Taking the file path input from the user 
# e.g. C:\Users\Bob\Desktop\example.txt
# or /home/Bob/Desktop/example.txt





def find_prod(x):
   
        #a = ('azoxystrobin')
        if re.search('R&D|RESEARCH|FREE SAMPLE|TESTING|SAMPLE|research|r&d|free|sample|For Lab use|lab use|LAB USE|FOR LAB USE',x) :
             return ""
        temp_products = list(set([i for i in products if i in re.split(' |\(|\)|\+|\\|/|\*|-|\s|\d',x.lower())]))
        if re.search('[2][-,\s]*[4][-\s]*[d]',x) :
             temp_products += [map_24d[re.search('[2][-,\s]*[4][-\s]*[d]',x)[0]]]
             #if re.search('AZOXY.{,50}|azoxy.{,50}',x):
              #temp_products += a
        if re.search('POTASH|potash|NPS|nps|PHOSPHATE|phosphate|SULFATE|sulfate|SULPHATE|sulphate',x):
             if len(temp_products)<1:
                    temp_products += 'fertilizer'
        if len(temp_products)>1 :
            temp_products = [i for i in temp_products if i!='pesticide']
            temp_products = [i for i in temp_products if i!='pesticides']
            temp_products = [i for i in temp_products if i!='rynaxypyr']
        return temp_products

    ##Mapping the Product Description column as string as sometimes it may not be read as a string
df2['Goods Description'] = df2['Goods Description'].map(str)
    
print("Working on Product Mapping")
temp = df2['Goods Description'].apply(lambda x: find_prod(x))
    # temp = temp.apply(lambda x:pesticide_check(x))

temp_processed = temp.apply(lambda x: ", ".join(x) if len(x)>=1 else "")
    
print("Product Mapping done , working on other conditions")

    ## Checking for the condition where UNit is GMS and quantity is one is not detected and anything with 0 quantity is also not detected
df2.loc[np.logical_and(df2['Unit'].gt('GMS'), np.less_equal(df2['Quantity'], 1)), 'Product'] = ''
df2.loc[df2["Quantity"]==0, "Product"] = ""
    
    ## Replacing the values that are returned as otherwise to the way we want it(The requirement is explained in the req doc)
temp_processed.replace('a, z, o, x, y, s, t, r, o, b, i, n','azoxystrobin',inplace = True)
temp_processed.replace('f, e, r, t, i, l, i, z, e, r','fertilizer',inplace = True)
temp_processed.replace('mancozebe','mancozeb',inplace = True)
temp_processed.replace('amoniusaltglyphosate71','glyphosate',inplace = True)
temp_processed.replace('AMONIUSALTGLYPHOSATE7171','glyphosate',inplace = True)
temp_processed.replace('rynaxypyr','chlorantraniliprole',inplace = True)
temp_processed.replace('glyphosatee','glyphosate',inplace = True)
temp_processed.replace('','Others',inplace = True)
    
df2['Product'] = temp_processed
    ## Filling up the blank values with 'Others'
df2['Product'] = df2['Product'].fillna('Others')
    
print("Product Mapping done , working on Suffix creation")
    
    #### CREATING THE SUFFIX COLUMN
    
# Creating a function for the same
def suf(x):
        temp_suffix = [] 
        if re.search('tech.{,50}|TECH.{,50}',x):
            temp_suffix = "TECH"
        elif re.search('IPA SALT|ipa salt',x):
            temp_suffix = "IPA SALT"
        elif re.search('62% IPA SALT|62% ipa salt',x):
            temp_suffix = "62% IPA SALT"
        elif re.search('41% SL|41% sl',x):
            temp_suffix = '41% SL'
        elif re.search('ALION PLUS.{50}|alion plus.{50}',x):
            temp_suffix = "ALION PLUS"
        elif re.search('95%',x):
            if len(list(df2['Product'].str.split(', '))) == 1:
                temp_suffix = "TECH"
            elif re.search('TC',x): 
                temp_suffix = "TECH"
            else :
                temp_suffix = "Others"  
        else :
            temp_suffix = "Others"
        return temp_suffix
    
    
df2['suffix'] =  df2['Goods Description'].apply(lambda x: suf(x))
print("Suffix Creation done")

    

Working on Product Mapping
Product Mapping done , working on other conditions
Product Mapping done , working on Suffix creation
Suffix Creation done


In [1]:
### Block for altering the entire data into the required format

temp2 = df2.drop(['Declaration No', 'HS Four Digit', 'HS Code','Destination Port','Indian Port','Address', 'City','Consinee Address','Port Code', 'IEC Phone', 'IEC Email', 'IEC Contact'],axis = 1)
temp2['Product Name'] = temp2['Product'] + " " + temp2['suffix']
temp2 = temp2.drop('suffix',axis = 1)
temp2['Type'] = 'Export'
temp2.rename(columns = {'Exporter Name':'IMPORTER/EXPORTER NAME','Country Origin':'Country','Goods Description':'PRODUCT DESCRIPTION','Quantity':'QTY','Unit':'UNIT','Unit Value Usd':'UNIT PRICE (USD)','Fob Usd':'Total_VALUE_USD','Currency':'INVOICE CURRENCY','Consinee Name':'SUPPLIER/BUYER NAME','Product':'Molecule Name'},inplace = True)


temp2 = temp2.loc[:,["Type","Date","IMPORTER/EXPORTER NAME","Country","PRODUCT DESCRIPTION","QTY","UNIT","UNIT PRICE (USD)","Total_VALUE_USD","INVOICE CURRENCY","SUPPLIER/BUYER NAME","Molecule Name","Product Name"]]
temp2['Product Name'].replace('Others Others','Others',inplace = True)

NameError: name 'df2' is not defined

In [ ]:
# Stacking the Export data into the main file

df_compiled = pd.concat([df_main,temp2],axis = 0 )

### --------------------------------------Taking input from the user for the Import file---------------------------------------------------------

In [2]:
# Taking the file path input from the user 

file_path = input('Enter the 38 Series Import file path: ')

# e.g. C:\Users\Bob\Desktop\example.txt
# or /home/Bob/Desktop/example.txt
print(file_path)

if os.path.exists(file_path):
    print('The file exists')

    with open(file_path, 'r', encoding="latin-1") as f:
        lines = f.readlines()

else:
    print('The specified file does NOT exist')



KeyboardInterrupt: Interrupted by user

In [ ]:
## Reading the file
df3= pd.read_excel(file_path)

In [ ]:
a = "hekdidh cjasdvh aduchc c9audc acuadc aca0spciasce423r 453o4646586 9.[vv 9 p;. ,k9 876t ghbnm, ./ '[p09 8765rwfvb3nm4k5lotg0b -[; ., ]]']"
a = a.split(' ')

In [ ]:
a

In [ ]:



def find_prod(x):
   
        #a = ('azoxystrobin')
        if re.search('R&D|RESEARCH|FREE SAMPLE|TESTING|SAMPLE|research|r&d|free|sample',x) :
             return ""
        temp_products = list(set([i for i in products if i in re.split(' |\(|\)|\+|\\|/|\*|-|\s|\d',x.lower())]))
      
        if re.search('[2][-,\s]*[4][-\s]*[d]',x) :
             temp_products += [map_24d[re.search('[2][-,\s]*[4][-\s]*[d]',x)[0]]]
             #if re.search('AZOXY.{,50}|azoxy.{,50}',x):
              #temp_products += a
        if re.search('POTASH|potash|NPS|nps|PHOSPHATE|phosphate|SULFATE|sulfate|SULPHATE|sulphate',x):
             if len(temp_products)<1:
                    temp_products += 'fertilizer'
        if len(temp_products)>1 :
            temp_products = [i for i in temp_products if i!='pesticide']
            temp_products = [i for i in temp_products if i!='pesticides']
            temp_products = [i for i in temp_products if i!='rynaxypyr']
        return temp_products

    ##Mapping the Product Description column as string as sometimes it may not be read as a string
df3['Goods Description'] = df3['Goods Description'].map(str)
    
print("Working on Product Mapping")
temp = df3['Goods Description'].apply(lambda x: find_prod(x))
    # temp = temp.apply(lambda x:pesticide_check(x))

temp_processed = temp.apply(lambda x: ", ".join(x) if len(x)>=1 else "")
    
print("Product Mapping done , working on other conditions")

    ## Checking for the condition where UNit is GMS and quantity is one is not detected and anything with 0 quantity is also not detected
df3.loc[np.logical_and(df3['Unit'].gt('GMS'), np.less_equal(df3['Quantity'], 1)), 'Product'] = ''
df3.loc[df3["Quantity"]==0, "Product"] = ""
    
    ## Replacing the values that are returned as otherwise to the way we want it(The requirement is explained in the req doc)
temp_processed.replace('a, z, o, x, y, s, t, r, o, b, i, n','azoxystrobin',inplace = True)
temp_processed.replace('f, e, r, t, i, l, i, z, e, r','fertilizer',inplace = True)
temp_processed.replace('mancozebe','mancozeb',inplace = True)
temp_processed.replace('amoniusaltglyphosate71','glyphosate',inplace = True)
temp_processed.replace('AMONIUSALTGLYPHOSATE7171','glyphosate',inplace = True)
temp_processed.replace('rynaxypyr','chlorantraniliprole',inplace = True)
temp_processed.replace('glyphosatee','glyphosate',inplace = True)
temp_processed.replace('','Others',inplace = True)
    
df3['Product'] = temp_processed
    ## Filling up the blank values with 'Others'
df3['Product'] = df3['Product'].fillna('Others')
    
print("Product Mapping done , working on Suffix creation")
    
    #### CREATING THE SUFFIX COLUMN
    
    # Creating a function for the same
def suf(x):
        temp_suffix = [] 
        if re.search('tech.{,50}|TECH.{,50}',x):
            temp_suffix = "TECH"
        elif re.search('IPA SALT|ipa salt',x):
            temp_suffix = "IPA SALT"
        elif re.search('62% IPA SALT|62% ipa salt',x):
            temp_suffix = "62% IPA SALT"
        elif re.search('41% SL|41% sl',x):
            temp_suffix = '41% SL'
        elif re.search('ALION PLUS.{50}|alion plus.{50}',x):
            temp_suffix = "ALION PLUS"
        elif re.search('95%',x):
            if len(list(df3['Product'].str.split(', '))) == 1:
                temp_suffix = "TECH"
            elif re.search('TC',x): 
                temp_suffix = "TECH"
            else :
                temp_suffix = "Others" 
        else :
            temp_suffix = "Others"
        return temp_suffix
    
    
df3['suffix'] =  df3['Goods Description'].apply(lambda x: suf(x))
print("Suffix Creation done")

    

In [294]:
### Block for altering the entire data int|o the required format


temp3 = df3.drop(['Port Code', 'Indian Port', 'Declaration No','Address', 'City', 'CHA No', 'CHA Name','HS Code','Total_Duty_Usd', 'Loading Port','Supplier Address', 'BE Type', 'IEC Phone','IEC Email', 'IEC Contact'],axis = 1)
temp3['Product Name'] = temp3['Product'] + " " + temp3['suffix']
temp3 = temp3.drop('suffix',axis = 1)
temp3['Type'] = 'Import'
temp3.rename(columns = {'Importer Name':'IMPORTER/EXPORTER NAME','Country Origin':'Country','Goods Description':'PRODUCT DESCRIPTION','Quantity':'QTY','Unit':'UNIT','Unit Price Usd':'UNIT PRICE (USD)','Total Value Usd':'Total_VALUE_USD','Inv Currency':'INVOICE CURRENCY','Supplier Name':'SUPPLIER/BUYER NAME','Product':'Molecule Name'},inplace = True)



temp3 = temp3.loc[:,["Type","Date","IMPORTER/EXPORTER NAME","Country","PRODUCT DESCRIPTION","QTY","UNIT","UNIT PRICE (USD)","Total_VALUE_USD","INVOICE CURRENCY","SUPPLIER/BUYER NAME","Molecule Name","Product Name"]]
temp3['Product Name'].replace('Others Others','Others',inplace = True)

In [441]:
# Stacking the Export data into the main file

df_final_compiled = pd.concat([df_compiled ,temp3],axis = 0 )

In [442]:
df_final_compiled = df_final_compiled[df_final_compiled['Molecule Name'] != 'Others']

In [443]:
df_final_compiled2 = df_final_compiled

In [444]:
df_final_compiled2['Country'].nunique()

206

In [445]:
df_final_compiled2['IMPORTER/EXPORTER NAME'].nunique()

4456

#### -------------------------------  WORKING ON THE COUNTRY, COMPANY , QTY and Molecule Name Columns  -------------------------

#### Working on the compnay column i,e IMPORTER/EXPORTER NAME

In [446]:
company = pd.read_excel(r'C:\Users\S1130866\Desktop\Product_Automate\Country and company.xlsx')
country = pd.read_excel(r'C:\Users\S1130866\Desktop\Product_Automate\Country and company.xlsx',sheet_name = "Country") 

In [447]:
unique = company.drop_duplicates()

In [448]:
unique_mismatches = unique[unique['Company'].str.lower() != unique['New'].str.lower()]

In [449]:
unique_mismatches.head()

,Company,New
8723,ADAMA INDIA PRIVATE LIMITED,ADAMA
29864,BASF CATALYSTS INDIA PRIVATE LIMITED,BASF
29889,BASF CHEMICALS INDIA PRIVATE LIMITED,BASF
29927,BASF INDIA LIMITED,BASF
30767,BAYER CROPSCIENCE LIMITED,Bayer


In [450]:
unique_mismatches.shape

(28, 2)

In [451]:
company_key_value = dict(zip(unique_mismatches['Company'].str.lower(),unique_mismatches['New'].str.lower()))

In [452]:
df_final_compiled2['IMPORTER/EXPORTER NAME'] = df_final_compiled2['IMPORTER/EXPORTER NAME'].str.lower().replace(company_key_value)

In [453]:
df_final_compiled2['IMPORTER/EXPORTER NAME'].head()

0    krishi rasayan exports private limited
1               willowood chemicals pvt ltd
2                                  syngenta
3                                  syngenta
4                                   crystal
Name: IMPORTER/EXPORTER NAME, dtype: object

In [454]:
df_final_compiled2['IMPORTER/EXPORTER NAME'].nunique()

4453

### Working on the country column

In [455]:
country.head()

,Country,New
0,AFGHANISTAN,Afghanistan
1,ALBANIA,Albania
2,ALGERIA,Algeria
3,ANGOLA,ANGOLA
4,ANGUILLA,ANGUILLA


In [456]:
unique2 = country.drop_duplicates()

In [457]:
unique_mismatches2 = unique2[unique2['Country'].str.lower() != unique2['New'].str.lower()]

In [458]:
unique_mismatches2.head()

,Country,New
21,"BOLIVIA, PLURINATIONAL STATE OF",Bolivia
41,"CONGO, THE DEMOCRATIC REPUBLIC OF THE",Congo
47,Democratic Republic of the Congo,Congo
72,GUINEA-BISSAU5,GUINEA-BISSAU
82,"IRAN, ISLAMIC REPUBLIC OF",Iran


In [459]:
unique_mismatches2.shape

(23, 2)

In [460]:
unique2 = country.drop_duplicates() 
unique_mismatches2 = unique2[unique2['Country'] != unique2['New']]
unique_mismatches2.shape
country_key_value = dict(zip(unique_mismatches2['Country'].str.lower(),unique_mismatches2['New'].str.lower()))
df_final_compiled2['Country'] = df_final_compiled2['Country'].str.lower().replace(country_key_value)

In [461]:
df_final_compiled2.head()

,Type,Date,IMPORTER/EXPORTER NAME,Country,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Molecule Name,Product Name,Avg_Price
0,Import,2021-01-15,krishi rasayan exports private limited,china,EMAMECTIN BENZOATE 95% TECHNICAL.ACTUAL USER R...,3000.0,KGS,168.000000,504000.0000,USD,HUNAN GOFAR FINE CHEMICAL INDUSTR,EMAMECTIN,EMAMECTIN TECH 95%,168.000000
1,Import,2021-01-28,willowood chemicals pvt ltd,china,EMAMECTIN BENZOATE 95% TECH (CIB NO. CIR-1976/...,5000.0,KGS,155.240198,776200.9898,CNY,WILLOWOOD HANGZHOU CO LIMITED,EMAMECTIN,EMAMECTIN TECH 95%,155.240198
2,Import,2021-02-05,syngenta,austria,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,1800.0,KGS,190.296299,342533.3379,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,EMAMECTIN Others,190.296299
3,Import,2021-02-19,syngenta,austria,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,2250.0,KGS,190.554503,428747.6312,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,EMAMECTIN Others,190.554503
4,Import,2021-03-04,crystal,china,EMAMECTIN BENZOATE TECHNICAL 95% (MIN) (CIB NO...,3000.0,KGS,228.500000,685500.0000,USD,HEBEI VEYONG BIO CHEMICAL CO LIMITED,EMAMECTIN,EMAMECTIN TECH 95%,228.500000


In [462]:
df_final_compiled3.head()

,Type,Date,IMPORTER/EXPORTER NAME,Country,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Molecule Name,Product Name,Avg_Price,AI
0,Import,2021-01-15,Krishi rasayan exports private limited,China,EMAMECTIN BENZOATE 95% TECHNICAL.ACTUAL USER R...,3.00,MT,168.000000,504000.0000,USD,HUNAN GOFAR FINE CHEMICAL INDUSTR,EMAMECTIN,EMAMECTIN TECH 95%,168.000000,emamectin tech 95%
1,Import,2021-01-28,Willowood chemicals pvt ltd,China,EMAMECTIN BENZOATE 95% TECH (CIB NO. CIR-1976/...,5.00,MT,155.240198,776200.9898,CNY,WILLOWOOD HANGZHOU CO LIMITED,EMAMECTIN,EMAMECTIN TECH 95%,155.240198,emamectin tech 95%
2,Import,2021-02-05,Syngenta,Austria,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,1.80,MT,190.296299,342533.3379,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,EMAMECTIN Others,190.296299,emamectin others
3,Import,2021-02-19,Syngenta,Austria,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,2.25,MT,190.554503,428747.6312,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,EMAMECTIN Others,190.554503,emamectin others
4,Import,2021-03-04,Crystal,China,EMAMECTIN BENZOATE TECHNICAL 95% (MIN) (CIB NO...,3.00,MT,228.500000,685500.0000,USD,HEBEI VEYONG BIO CHEMICAL CO LIMITED,EMAMECTIN,EMAMECTIN TECH 95%,228.500000,emamectin tech 95%


#### --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [463]:
df_final_compiled2['IMPORTER/EXPORTER NAME'].nunique()

4453

In [464]:
df_final_compiled2['Country'].nunique()

188

In [465]:
df_final_compiled2.to_csv(r"C:\Users\S1130866\Desktop\6thMarch.csv")

In [466]:
df_final_compiled2.isna().sum()

Type                        0
Date                        0
IMPORTER/EXPORTER NAME      0
Country                     0
PRODUCT DESCRIPTION         0
QTY                         0
UNIT                        0
UNIT PRICE (USD)            0
Total_VALUE_USD             0
INVOICE CURRENCY            0
SUPPLIER/BUYER NAME       115
Molecule Name               0
Product Name                0
Avg_Price                   0
dtype: int64

### -------------------------------  UNIT COLUMN -------------------------------------------

In [467]:
df_final_compiled3 = df_final_compiled2

In [468]:
# df_final_compiled3['UNIT'] =df_final_compiled3['UNIT'].str.rstrip().replace(unit_dict)

In [469]:
df_final_compiled['UNIT'].nunique()

45

In [470]:
df_final_compiled3['UNIT'].unique()

array(['KGS', 'LTR', 'KGS ', 'LTR ', 'LT/KG', 'NOS ', 'PCS ', 'MTS ',
       'MGS ', 'SET ', 'NOS', 'UNT ', 'VLS ', 'GMS ', 'MTR ', 'PAC',
       'PCS', 'LBS ', 'GMS', 'MTS', 'CAN', 'BTL', 'RLS', 'DRM', 'TUB',
       'MLT', 'GRS ', 'KIT', 'CBM ', 'SQM ', 'MGS', 'UNT', 'SET', 'LBS',
       'VLS', 'THD', 'MTR', 'CTM', 'PRS', 'KGA', 'DOZ', 'BAG', 'KLR',
       'TBS', 'BRL'], dtype=object)

In [471]:
df_final_compiled3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165599 entries, 0 to 51938
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Type                    165599 non-null  object        
 1   Date                    165599 non-null  datetime64[ns]
 2   IMPORTER/EXPORTER NAME  165599 non-null  object        
 3   Country                 165599 non-null  object        
 4   PRODUCT DESCRIPTION     165599 non-null  object        
 5   QTY                     165599 non-null  float64       
 6   UNIT                    165599 non-null  object        
 7   UNIT PRICE (USD)        165599 non-null  float64       
 8   Total_VALUE_USD         165599 non-null  float64       
 9   INVOICE CURRENCY        165599 non-null  object        
 10  SUPPLIER/BUYER NAME     165484 non-null  object        
 11  Molecule Name           165599 non-null  object        
 12  Product Name            165599 

In [472]:
df_final_compiled3['UNIT'] = df_final_compiled3['UNIT'].apply(lambda x: x.strip())

df_final_compiled3.loc[df_final_compiled3['QTY'] == 'KGS', 'QTY'] = 100* df_final_compiled3['QTY']

df_final_compiled3.where('UNIT' == 'KGS', df_final_compiled3['QTY']* 0.001 )

In [473]:
'''

'KGS':'KG/LT',
'KGS':'KG/LT,
'GMS':'GMs/ML',
'GMS':'GMs/ML',
'LT/KG':'KG/LT'
'LTR':'KG/LT',
'LTR':'KG/LT',
'MTS':'Metric Tons',
'MTS':'Metric Tons',
'MLT':'Metric LT'
'MGS':'GMs/ML',
'MGS':'GMs/ML'

'''

#1l = 1kg


df_final_compiled3['QTY'] = df_final_compiled3.apply(lambda x: x['QTY'] * 0.001 if x['UNIT']== 'KGS' else x['QTY'], axis=1)
df_final_compiled3['QTY'] = df_final_compiled3.apply(lambda x: x['QTY'] * 0.001 if x['UNIT']== 'LTR' else x['QTY'], axis=1)
df_final_compiled3['QTY'] = df_final_compiled3.apply(lambda x: x['QTY'] * 0.001 if x['UNIT']== 'LT/KG' else x['QTY'], axis=1)
df_final_compiled3['QTY'] = df_final_compiled3.apply(lambda x: x['QTY'] * 1 if x['UNIT']== 'MTS' else x['QTY'], axis=1)
df_final_compiled3['QTY'] = df_final_compiled3.apply(lambda x: x['QTY'] * 1 if x['UNIT']== 'MLT' else x['QTY'], axis=1)
df_final_compiled3['QTY'] = df_final_compiled3.apply(lambda x: x['QTY'] * 0.000001 if x['UNIT']== 'MGS' else x['QTY'], axis=1)
df_final_compiled3['QTY'] = df_final_compiled3.apply(lambda x: x['QTY'] * 0.000001 if x['UNIT']== 'GMS' else x['QTY'], axis=1)
df_final_compiled3['QTY'] = df_final_compiled3.apply(lambda x: x['QTY'] * 0.000453592 if x['UNIT']== 'LBS' else x['QTY'], axis=1)





df_final_compiled3['QTY'] = df_final_compiled3.apply(lambda x: x['QTY'] * 0.001 if x['UNIT']== 'KGS' else x['QTY'], axis=1)
elif df_final_compiled3['UNIT'] == 'GMS':
    df_final_compiled3['QTY'] = df_final_compiled3['QTY'] * 0.000001
elif df_final_compiled3['UNIT'].str.rstrip().any() == 'LT/KG':
    df_final_compiled3['QTY'] = df_final_compiled3['QTY'] * 0.001
elif df_final_compiled3['UNIT'].str.rstrip().any() == 'LTR':
    df_final_compiled3['QTY'] = df_final_compiled3['QTY'] * 0.001
elif df_final_compiled3['UNIT'].str.rstrip().any() == 'MTS':
    df_final_compiled3['QTY'] = df_final_compiled3['QTY'] * 1
elif df_final_compiled3['UNIT'].str.rstrip().any() == 'MLT':
    df_final_compiled3['QTY'] = df_final_compiled3['QTY'] * 1
elif df_final_compiled3['UNIT'].str.rstrip().any() == 'MGS':
    df_final_compiled3['QTY'] = df_final_compiled3['QTY'] * 0.000001
    


In [474]:
## Values to be converted into the required format 

"""

'KGS':'KG/LT',
'KGS':'KG/LT ,
'GMS':'GMs/ML' ,
'GMS':'GMs/ML' ,
'LT/KG':'KG/LT'
'LTR':'KG/LT',
'LTR':'KG/LT',
'MTS':'Metric Tons' ,
'MTS':'Metric Tons' ,
'MLT':'Metric LT '
'MGS':'GMs/ML' ,
'MGS':'GMs/ML'


"""



unit_dict_Other = {'BAG':'Other',
             'BTL':'Other', 
             'CAN':'Other',
             'CBM':'Other',
             'CTM':'Other',
             'DOZ':'Other',
             'DRM':'Other',
'GRS':'Other',
'KGA':'Other',
'KIT':'Other',
'KLR':'Other',
'MTR':'Other',
'MTR':'Other',
'NOS':'Other',
'NOS':'Other',
'PAC':'Other',
'PCS':'Other',
'PCS':'Other',
'PRS':'Other',
'RLS':'Other',
'SET':'Other',
'SET':'Other',
'SQM':'Other',
'THD':'Other',
'TUB':'Other',
'UNT':'Other',
'UNT':'Other',
'VLS':'Other',
'VLS':'Other',
'TBS':'Other',
'BRL':'Other',
'KGS':'KG/LT',
'KGS':'KG/LT',
'GMS':'GMs/ML',
'GMS':'GMs/ML',
'LT/KG':'KG/LT',
'LTR':'KG/LT',
'LTR':'KG/LT',
'MTS':'Metric Tons',
'MTS':'Metric Tons',
'MLT':'Metric LT',
'MGS':'GMs/ML',
'MGS':'GMs/ML',
'KGS':'MT',
'KGS':'MT',
'GMS':'MT',
'GMS':'MT',
'LT/KG':'MT',
'LTR':'MT',
'LTR':'MT',
'MTS':'MT',
'MTS':'MT',
'MLT':'MT',
'MGS':'MT',
'MGS':'MT',
'LBS':'MT'
}

In [475]:
df_final_compiled3['UNIT'] = df_final_compiled3['UNIT'].apply(lambda x: x.strip())

In [476]:
df_final_compiled3['UNIT'] = df_final_compiled3['UNIT'].map(unit_dict_Other)

###  ----------- ORDER FOR AI's  --------------------

In [477]:
AI_Order = pd.read_excel(r"C:\Users\S1130866\Desktop\Product_Automate\Tech vs Other.xlsx")

In [478]:
AI_Order.head()

,Product Name 1,Product Name 2
0,abamectin Others,abamectin Others
1,abamectin TECH,abamectin TECH
2,"abamectin, emamectin Others",Combination
3,acephate Others,acephate Others
4,acephate TECH,acephate TECH


In [479]:
AI_Mapping = dict(zip(AI_Order['Product Name 1'].str.lower(),AI_Order['Product Name 2'].str.lower()))

In [480]:
AI_Mapping

{'abamectin others': 'abamectin others',
 'abamectin tech': 'abamectin tech',
 'abamectin, emamectin others': 'combination',
 'acephate others': 'acephate others',
 'acephate tech': 'acephate tech',
 'acephate, bifenthrin others': 'combination',
 'acephate, imidacloprid others': 'combination',
 'acequinocyl tech': 'acequinocyl tech',
 'acetamipird others': 'acetamipird others',
 'acetamiprid others': 'acetamiprid others',
 'acetamiprid tech': 'acetamiprid tech',
 'acetamiprid, acitamiprid others': 'combination',
 'acetamiprid, bifenthrin others': 'combination',
 'acetamiprid, buprofenzin others': 'combination',
 'acetamiprid, cyhalothrin, lambda others': 'combination',
 'acetamiprid, cypermethrin others': 'combination',
 'acetamiprid, emamectin others': 'combination',
 'acetamiprid, fipronil others': 'combination',
 'acetamiprid, indoxacarb others': 'combination',
 'acetamiprid, lambda others': 'combination',
 'acetanilide others': 'acetanilide others',
 'acetochlor others': 'acetochlo

def AI_MAP(x) :
    a = 'Others'
    if x in AI_Mapping.keys():
        x.map(AI_Mapping)
    else  : 
        elements =  [i for i in x.split(', ')]
        if len(elements)>1:
            return str(a)
        else : 
            return x

In [481]:
df_final_compiled3['AI'] = df_final_compiled3['Product Name'].apply(lambda x : x.lower())

df_final_compiled3['AI'] = df_final_compiled3['AI'].map(AI_Mapping).fillna(df_final_compiled3['Product Name'])



In [482]:
df_final_compiled3['AI'].head()

0    emamectin tech 95%
1    emamectin tech 95%
2      emamectin others
3      emamectin others
4    emamectin tech 95%
Name: AI, dtype: object

In [483]:
df_final_compiled3['AI'].value_counts()

not relevant                35831
mancozeb others             15867
calcium others               8281
combination                  8118
control others               7161
                            ...  
acetamipird others              1
mefentrifluconazole tech        1
mevinphos others                1
asulam, upl others              1
epoxiconazole Others            1
Name: AI, Length: 1173, dtype: int64

In [484]:
df_final_compiled3['Country'] = df_final_compiled3['Country'].str.capitalize()
df_final_compiled3['IMPORTER/EXPORTER NAME'] = df_final_compiled3['IMPORTER/EXPORTER NAME'].str.capitalize()

In [485]:
df_final_compiled3.head()

,Type,Date,IMPORTER/EXPORTER NAME,Country,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Molecule Name,Product Name,Avg_Price,AI
0,Import,2021-01-15,Krishi rasayan exports private limited,China,EMAMECTIN BENZOATE 95% TECHNICAL.ACTUAL USER R...,3.00,MT,168.000000,504000.0000,USD,HUNAN GOFAR FINE CHEMICAL INDUSTR,EMAMECTIN,EMAMECTIN TECH 95%,168.000000,emamectin tech 95%
1,Import,2021-01-28,Willowood chemicals pvt ltd,China,EMAMECTIN BENZOATE 95% TECH (CIB NO. CIR-1976/...,5.00,MT,155.240198,776200.9898,CNY,WILLOWOOD HANGZHOU CO LIMITED,EMAMECTIN,EMAMECTIN TECH 95%,155.240198,emamectin tech 95%
2,Import,2021-02-05,Syngenta,Austria,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,1.80,MT,190.296299,342533.3379,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,EMAMECTIN Others,190.296299,emamectin others
3,Import,2021-02-19,Syngenta,Austria,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,2.25,MT,190.554503,428747.6312,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,EMAMECTIN Others,190.554503,emamectin others
4,Import,2021-03-04,Crystal,China,EMAMECTIN BENZOATE TECHNICAL 95% (MIN) (CIB NO...,3.00,MT,228.500000,685500.0000,USD,HEBEI VEYONG BIO CHEMICAL CO LIMITED,EMAMECTIN,EMAMECTIN TECH 95%,228.500000,emamectin tech 95%


In [486]:
df_final_compiled4 = df_final_compiled3.loc[df_final_compiled3['UNIT'] == 'MT']

In [487]:
df_final_compiled4.shape

(119732, 15)

In [488]:
import seaborn as sns

In [489]:
df_final_compiled4['QTY'].describe()

count    119732.000000
mean         23.424790
std         435.012424
min           0.000000
25%           1.440000
50%          12.800000
75%          20.000000
max      100000.000000
Name: QTY, dtype: float64

In [490]:
df_final_compiled3.to_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Compiled_File\Main.xlsx",sheet_name = 'Data')

In [491]:
df_final_compiled3.head()

,Type,Date,IMPORTER/EXPORTER NAME,Country,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Molecule Name,Product Name,Avg_Price,AI
0,Import,2021-01-15,Krishi rasayan exports private limited,China,EMAMECTIN BENZOATE 95% TECHNICAL.ACTUAL USER R...,3.00,MT,168.000000,504000.0000,USD,HUNAN GOFAR FINE CHEMICAL INDUSTR,EMAMECTIN,EMAMECTIN TECH 95%,168.000000,emamectin tech 95%
1,Import,2021-01-28,Willowood chemicals pvt ltd,China,EMAMECTIN BENZOATE 95% TECH (CIB NO. CIR-1976/...,5.00,MT,155.240198,776200.9898,CNY,WILLOWOOD HANGZHOU CO LIMITED,EMAMECTIN,EMAMECTIN TECH 95%,155.240198,emamectin tech 95%
2,Import,2021-02-05,Syngenta,Austria,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,1.80,MT,190.296299,342533.3379,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,EMAMECTIN Others,190.296299,emamectin others
3,Import,2021-02-19,Syngenta,Austria,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,2.25,MT,190.554503,428747.6312,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,EMAMECTIN Others,190.554503,emamectin others
4,Import,2021-03-04,Crystal,China,EMAMECTIN BENZOATE TECHNICAL 95% (MIN) (CIB NO...,3.00,MT,228.500000,685500.0000,USD,HEBEI VEYONG BIO CHEMICAL CO LIMITED,EMAMECTIN,EMAMECTIN TECH 95%,228.500000,emamectin tech 95%
